# Credit card fraud detector

## Setup AWS Credentials in the environment

[AWS configuration instructions](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html)

## Investigate and process the data

Let's start by downloading and reading in the credit card fraud data set.

In [16]:
%%bash
# wget https://s3-us-west-2.amazonaws.com/sagemaker-e2e-solutions/fraud-detection/creditcardfraud.zip
wget https://ml-in-production-book-code.s3-us-west-1.amazonaws.com/creditcardfraud.zip
unzip creditcardfraud.zip

Archive:  creditcardfraud.zip


--2020-03-01 07:19:07--  https://ml-in-production-book-code.s3-us-west-1.amazonaws.com/creditcardfraud.zip
Resolving ml-in-production-book-code.s3-us-west-1.amazonaws.com (ml-in-production-book-code.s3-us-west-1.amazonaws.com)... 52.219.116.65
Connecting to ml-in-production-book-code.s3-us-west-1.amazonaws.com (ml-in-production-book-code.s3-us-west-1.amazonaws.com)|52.219.116.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69155632 (66M) [application/zip]
Saving to: ‘creditcardfraud.zip.1’

     0K .......... .......... .......... .......... ..........  0% 1.73M 38s
    50K .......... .......... .......... .......... ..........  0% 1.43M 42s
   100K .......... .......... .......... .......... ..........  0% 1.11M 48s
   150K .......... .......... .......... .......... ..........  0% 1.74M 45s
   200K .......... .......... .......... .......... ..........  0% 3.00M 41s
   250K .......... .......... .......... .......... ..........  0% 3.73M 37s
   300K .....

In [17]:
import numpy as np 
import pandas as pd

data = pd.read_csv('creditcard.csv', delimiter=',')

Let's take a peek at our data (we only show a subset of the columns in the table):

In [18]:
print(data.columns)
data[['Time', 'V1', 'V2', 'V15', 'V27', 'V28', 'Amount', 'Class']].describe()

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')


,Time,V1,V2,V15,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.165980e-15,3.416908e-16,4.913003e-15,-3.666453e-16,-1.220404e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,9.153160e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.498945e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-5.828843e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,4.807155e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,6.488208e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,8.877742e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


The class column corresponds to whether or not a transaction is fradulent. We see that the majority of data is non-fraudulant with only $492$ ($.173\%$) of the data corresponding to fraudulant examples.

In [19]:
nonfrauds, frauds = data.groupby('Class').size()
print('Number of frauds: ', frauds)
print('Number of non-frauds: ', nonfrauds)
print('Percentage of fradulent data:', 100.*frauds/(frauds + nonfrauds))

Number of frauds:  492
Number of non-frauds:  284315
Percentage of fradulent data: 0.1727485630620034


This dataset has 28 columns, $V_i$ for $i=1..28$ of anonymized features along with columns for time, amount, and class. We already know that the columns $V_i$ have been normalized to have $0$ mean and unit standard deviation as the result of a PCA. You can read more about PCA here:. 

Tip: For our dataset this amount of preprocessing will give us reasonable accuracy, but it's important to note that there are more preprocessing steps one can use to improve accuracy . For unbalanced data sets like ours where the positive (fraudulent) examples occur much less frequently than the negative (legitimate) examples, we may try “over-sampling” the minority dataset by generating synthetic data (read about SMOTE in Data Mining for Imbalanced Datasets: An Overview (https://link.springer.com/chapter/10.1007%2F0-387-25465-X_40) or undersampling the majority class by using ensemble methods (see http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.68.6858&rep=rep1&type=pdfor).

In [20]:
feature_columns = data.columns[:-1]
label_column = data.columns[-1]
features = data[feature_columns].values.astype('float32')
labels = (data[label_column].values).astype('float32')

Let's do some analysis and discuss different ways we can preprocess our data. Let's discuss the way in which this data was preprocessed.

## SageMaker Linear Learner

### Prepare Data and Upload to S3

The Amazon common libraries provide utilities to convert NumPy n-dimensional arrays into a the Record-IO format which SageMaker uses for a concise representation of features and labels. The Record-IO format is implemented via protocol buffer so the serialization is very efficient.

In [21]:
import io
import sagemaker.amazon.common as smac

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, features, labels)
buf.seek(0);

Now we upload the data to S3 using boto3.

In [23]:
import boto3
import os
import sagemaker

session = sagemaker.Session()
bucket = "ml-in-production-book-code"

prefix = 'linear-learner'
key = 'recordio-pb-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)

s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('Uploaded training data location: {}'.format(s3_train_data))

output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('Training artifacts will be uploaded to: {}'.format(output_location))

ClientError: An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.

## Train SageMaker Linear Learner 

Now we train a Linear Learner using SageMaker's built-in algorithm. To specify the Linear Learner algorithm, we use a utility function to obtain it's URI. A complete list of build-in algorithms is found here: https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html

In [16]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, 'linear-learner')

632365934929.dkr.ecr.us-west-1.amazonaws.com/linear-learner:1


SageMaker abstracts training with Estimators. We can pass container, and all parameters to the estimator, as well as the hyperparameters for the linear learner and fit the estimator to the data in S3.

In [42]:
from sagemaker import get_execution_role

sagemaker_role = 'arn:aws:iam::321459297935:role/service-role/AmazonSageMaker-ExecutionRole-20200111T200041'

linear = sagemaker.estimator.Estimator(container,
                                       sagemaker_role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.c4.xlarge',
                                       output_path=output_location,
                                       sagemaker_session=session)
linear.set_hyperparameters(feature_dim=features.shape[1],
                           predictor_type='binary_classifier',
                           mini_batch_size=200)

linear.fit({'train': s3_train_data})

2020-02-10 04:43:04 Starting - Starting the training job...
2020-02-10 04:43:05 Starting - Launching requested ML instances...
2020-02-10 04:44:04 Starting - Preparing the instances for training.........
2020-02-10 04:45:28 Downloading - Downloading input data
2020-02-10 04:45:28 Training - Downloading the training image.Docker entrypoint called with argument(s): train
[02/10/2020 04:45:44 INFO 139835106260800] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u'aut

[2020-02-10 04:47:03.898] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 39750, "num_examples": 1425, "num_bytes": 47847576}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.007420676830872172, "sum": 0.007420676830872172, "min": 0.007420676830872172}}, "EndTime": 1581310023.899073, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1581310023.898988}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005714684001750867, "sum": 0.005714684001750867, "min": 0.005714684001750867}}, "EndTime": 1581310023.899178, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1581310023.899157}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00764150966

[2020-02-10 04:47:43.194] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 39286, "num_examples": 1425, "num_bytes": 47847576}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006549438021768387, "sum": 0.006549438021768387, "min": 0.006549438021768387}}, "EndTime": 1581310063.194204, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1581310063.194115}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005025194310171142, "sum": 0.005025194310171142, "min": 0.005025194310171142}}, "EndTime": 1581310063.194286, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1581310063.194266}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00668306904

[2020-02-10 04:48:22.242] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 39041, "num_examples": 1425, "num_bytes": 47847576}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006084246800583264, "sum": 0.006084246800583264, "min": 0.006084246800583264}}, "EndTime": 1581310102.242532, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1581310102.24244}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004734352222582315, "sum": 0.004734352222582315, "min": 0.004734352222582315}}, "EndTime": 1581310102.242623, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1581310102.242607}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006080260812

[2020-02-10 04:49:01.358] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 39109, "num_examples": 1425, "num_bytes": 47847576}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005935612929987543, "sum": 0.005935612929987543, "min": 0.005935612929987543}}, "EndTime": 1581310141.358538, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1581310141.35845}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00456818941899502, "sum": 0.00456818941899502, "min": 0.00456818941899502}}, "EndTime": 1581310141.358628, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1581310141.358607}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00581366891348

...

[2020-02-10 04:53:34.397] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 25, "duration": 39300, "num_examples": 1425, "num_bytes": 47847576}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0056820790946622264, "sum": 0.0056820790946622264, "min": 0.0056820790946622264}}, "EndTime": 1581310414.398039, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1581310414.39795}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004223927885563417, "sum": 0.004223927885563417, "min": 0.004223927885563417}}, "EndTime": 1581310414.398135, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1581310414.398116}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004612

[2020-02-10 04:54:13.624] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 39221, "num_examples": 1425, "num_bytes": 47847576}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005650635345752633, "sum": 0.005650635345752633, "min": 0.005650635345752633}}, "EndTime": 1581310453.62477, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1581310453.624682}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004216853727869757, "sum": 0.004216853727869757, "min": 0.004216853727869757}}, "EndTime": 1581310453.624861, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1581310453.624846}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004541245

[2020-02-10 04:54:52.960] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 39331, "num_examples": 1425, "num_bytes": 47847576}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005620591469178384, "sum": 0.005620591469178384, "min": 0.005620591469178384}}, "EndTime": 1581310492.960832, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 13}, "StartTime": 1581310492.960742}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004178281945923312, "sum": 0.004178281945923312, "min": 0.004178281945923312}}, "EndTime": 1581310492.960924, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 13}, "StartTime": 1581310492.960906}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00448250

[2020-02-10 04:55:31.906] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 31, "duration": 38940, "num_examples": 1425, "num_bytes": 47847576}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005591145085172173, "sum": 0.005591145085172173, "min": 0.005591145085172173}}, "EndTime": 1581310531.906411, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 14}, "StartTime": 1581310531.906326}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00414762033569153, "sum": 0.00414762033569153, "min": 0.00414762033569153}}, "EndTime": 1581310531.906505, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 14}, "StartTime": 1581310531.906484}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00443457874

[2020-02-10 04:55:38.026] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 38, "duration": 3597, "num_examples": 1425, "num_bytes": 47847576}
[02/10/2020 04:55:38 INFO 139835106260800] #train_score (algo-1) : ('binary_classification_cross_entropy_objective', 0.004155472405855248)
[02/10/2020 04:55:38 INFO 139835106260800] #train_score (algo-1) : ('binary_classification_accuracy', 0.9993469261640339)
[02/10/2020 04:55:38 INFO 139835106260800] #train_score (algo-1) : ('binary_f_1.000', 0.7951541850220264)
[02/10/2020 04:55:38 INFO 139835106260800] #train_score (algo-1) : ('precision', 0.8677884615384616)
[02/10/2020 04:55:38 INFO 139835106260800] #train_score (algo-1) : ('recall', 0.733739837398374)
[02/10/2020 04:55:38 INFO 139835106260800] #quality_metric: host=algo-1, train binary_classification_cross_entropy_objective <loss>=0.00415547240586
[02/10/2020 04:55:38 INFO 139835106260800] #quality_metric: host=algo-1, train binary_classification_accurac

### Host Linear Classifier

Now we deploy the estimator to and endpoint.

In [43]:
from sagemaker.predictor import csv_serializer, json_deserializer

linear_predictor = linear.deploy(initial_instance_count=1,
                                 endpoint_name="fraud-detection-endpoint",
                                 instance_type='ml.m4.xlarge')
# Specify input and output formats.
linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

------------!

## Clean up

We will leave the prediction endpoint running at the end of this notebook so we can handle incoming event streams. However, don't forget to delete the prediction endpoint when you're done. You can do that at the Amazon SageMaker console in the Endpoints page. Or you can run `linear_predictor.delete_endpoint()`

In [44]:
linear_predictor.delete_endpoint()


## Data Acknowledgements

The dataset used to demonstrated the fraud detection solution has been collected and analysed during a research collaboration of Worldline and the Machine Learning Group (http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection. More details on current and past projects on related topics are available on https://www.researchgate.net/project/Fraud-detection-5 and the page of the [DefeatFraud](https://mlg.ulb.ac.be/wordpress/portfolio_page/defeatfraud-assessment-and-validation-of-deep-feature-engineering-and-learning-solutions-for-fraud-detection/) project
We cite the following works:
* Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015
* Dal Pozzolo, Andrea; Caelen, Olivier; Le Borgne, Yann-Ael; Waterschoot, Serge; Bontempi, Gianluca. Learned lessons in credit card fraud detection from a practitioner perspective, Expert systems with applications,41,10,4915-4928,2014, Pergamon
* Dal Pozzolo, Andrea; Boracchi, Giacomo; Caelen, Olivier; Alippi, Cesare; Bontempi, Gianluca. Credit card fraud detection: a realistic modeling and a novel learning strategy, IEEE transactions on neural networks and learning systems,29,8,3784-3797,2018,IEEE
* Dal Pozzolo, Andrea Adaptive Machine learning for credit card fraud detection ULB MLG PhD thesis (supervised by G. Bontempi)
* Carcillo, Fabrizio; Dal Pozzolo, Andrea; Le Borgne, Yann-Aël; Caelen, Olivier; Mazzer, Yannis; Bontempi, Gianluca. Scarff: a scalable framework for streaming credit card fraud detection with Spark, Information fusion,41, 182-194,2018,Elsevier
* Carcillo, Fabrizio; Le Borgne, Yann-Aël; Caelen, Olivier; Bontempi, Gianluca. Streaming active learning strategies for real-life credit card fraud detection: assessment and visualization, International Journal of Data Science and Analytics, 5,4,285-300,2018,Springer International Publishing